In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns; sns.set(style="ticks", color_codes=True)
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import seaborn as sns
import cufflinks as cf
import plotly.express as px
%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected = True)
cf.go_offline() 



from wordcloud import WordCloud  

#for text processing
import string
import re
import nltk
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#for feature selection
from sklearn import decomposition

#for model building
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import warnings
warnings.filterwarnings('ignore')


from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.colors import n_colors
from plotly.subplots import make_subplots

init_notebook_mode(connected=True)
import cufflinks as cf
cf.go_offline()
from wordcloud import WordCloud , ImageColorGenerator

from nltk.corpus import stopwords
from textblob import TextBlob

In [ ]:

df = pd.read_csv('/kaggle/input/all-trumps-twitter-insults-20152021/trump_insult_tweets_2014_to_2021.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.info

In [ ]:
df.isnull().sum()

In [ ]:
df=df.dropna(axis=0)

In [ ]:
from datetime import datetime

df['date'] = pd.to_datetime(df['date'], infer_datetime_format = True)
df['year']=df['date'].dt.year
df['month']=df['date'].dt.month
df['week']=df['date'].dt.week
df.head()

In [ ]:
df_year = pd.DataFrame(df.groupby('year')['insult'].count().reset_index())
df_year.columns=['year','count']
df_year = df_year.sort_values(['count'],ascending=True)

fig = px.bar(df_year,y='year',x='count',
             orientation='h',color='count')
fig.update_layout(title_text='number of insults in each year',title_x=0.5,
                  template='plotly_dark')
fig.show()

In [ ]:
df_2020=df[df.year==2020]
most_Insulted = df_2020.groupby(['target'])['insult'].count().reset_index(name='count').sort_values(by='count',ascending=False)


fig = px.bar(most_Insulted.head(10),y='count',x='target',color='count')
fig.update_layout(title_text='most instulted in year 2020',title_x=0.5,template='plotly_dark')
fig.show()

In [ ]:
most_Insulted = df.groupby(['target'])['Unnamed: 0'].count().reset_index(name='count').sort_values(by='count',ascending=False)

fig_dims = (15, 8)
fig, ax1 = plt.subplots(figsize=fig_dims)
figure1= sns.barplot(x='target',y='count' , ax=ax1 , data=most_Insulted.head(25))
for item in figure1.get_xticklabels():
    item.set_rotation(90)

ax = plt.gca()

# Iterate through the list of axes' patches
for p in ax.patches:
    ax.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), 
            fontsize=12, color='black', ha='center', va='bottom')

plt.title('most-Insulted targets',size='20') 

plt.show()

In [ ]:
df_target = pd.DataFrame(df.groupby('target')['insult'].count().reset_index())
df_target.columns=['target','count']
df_target = df_target.sort_values(['count'],ascending=False)
df_target

In [ ]:
target = df_target['target'].iloc[0:10,].tolist()
count=df_target['count'].iloc[0:10,].tolist()

fig = go.Figure(data=[go.Table(header=dict(values=['target','Number of insults'],
                                          fill_color='blue',
                                           height=30),
                 cells=dict(values=[target,count],
                            height=30))
                     ])
fig.update_layout(title_text='Top 10 targets with most number of insults',title_x=0.5,
                  template='plotly_dark')
fig.show()

In [ ]:
unique_targetcounts = df.target.value_counts()
unique_targetcounts

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

target_text = " ".join(df.target)
wordcloud = WordCloud(width=1200, height=600).generate(text=target_text)
plt.figure(figsize=(25,18))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
insult_text = " ".join(df.insult)
wordcloud = WordCloud(width=1200, height=700).generate(text=insult_text)
plt.figure(figsize=(25,18))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

Data Cleaning

In [ ]:
#lowercase the column names
df.columns = df.columns.str.lower()

In [ ]:
df.columns

In [ ]:
df1=df[['date', 'target', 'insult', 'tweet', 'year']]

In [ ]:
df['tweet'] = df['tweet'].astype('str')
df['insult'] = df['insult'].astype('str')

In [ ]:
def clean_data(text):
    text = text.lower()  # convert all the text into lowercase
    text = text.strip()  #remove starting and trailing whitespaces
    special_char_reg = '([a-zA-Z0-9]+)' + '[!"#$%&\'()*+,-./:;<=>?@\\^_`{|}~]' + '([a-zA-Z0-9]+)'
    text = re.sub(special_char_reg, ' ', text)
    text = re.sub(r'\s+', ' ', text) #remove all line formattings
    text = re.sub(r'\d+', '', text) #remove digits
    text = re.sub(r'\w*\d\w*', '', text)
    text = re.sub(r"\w+…|…", "", text)  # Remove ellipsis (and last word)
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)
    text = ''.join(c for c in text if c not in string.punctuation)   #remove pecial symbols from job titles
    return text

In [ ]:
tweet_df = df.tweet.apply(lambda x : clean_data(x))
tweet_df.head()

In [ ]:
insult_df = df.insult.apply(lambda x : clean_data(x))
insult_df.head()

In [ ]:
from nltk import WordNetLemmatizer
def lemma(text):
    word_list = nltk.word_tokenize(text) #tokenize beofre lemmatization
    lemma_output = ' '.join(WordNetLemmatizer().lemmatize(word) for word in word_list)
    return lemma_output
tweet_df_lemma = tweet_df.apply(lambda x : lemma(x))
insult_df_lemma = insult_df.apply(lambda x : lemma(x))

In [ ]:

from nltk import word_tokenize
def remove_stopwords_and_tokenize(text):
    the_stopwords = set(stopwords.words("english"))
    tokens = word_tokenize(text)  # tokenize 
    tokens = [t for t in tokens if not t in the_stopwords]  
    tokens = [t for t in tokens if len(t) > 1] 
    return tokens

In [ ]:
tweet_df_token = tweet_df_lemma.apply(lambda x : remove_stopwords_and_tokenize(x))
tweet_df_token.head()

In [ ]:
insult_df_token = insult_df_lemma.apply(lambda x : remove_stopwords_and_tokenize(x))
insult_df_token.head()

In [ ]:
#Tokenization using count vectorizer
count_vect = CountVectorizer(ngram_range=(1,1))
token = count_vect.fit_transform(tweet_df_lemma)
token

In [ ]:
print('Total number of tokens/words in all the tweets - ', len(count_vect.get_feature_names()))

# Top Insults

In [ ]:
#Tokenization using count vectorizer
count_vect = CountVectorizer(ngram_range=(1,1))
token = count_vect.fit_transform(insult_df_lemma)
token

In [ ]:
temp_df =  pd.DataFrame(token.toarray(), columns=count_vect.get_feature_names())
temp_df.tail()

In [ ]:
#count the accurence of each token in entire corpus
count_df_insult = temp_df.apply(lambda x : x.sum())

In [ ]:
count_df_insult = pd.DataFrame(count_df_insult).reset_index()
count_df_insult.columns = ['Word', 'Count']
most_insults= count_df_insult.sort_values(by= 'Count', ascending=False)
most_insults.head(10)

In [ ]:
fig = px.bar(count_df_insult.head(30),y='Word',x='Count',
             orientation='h',color='Count')
fig.update_layout(title_text='most used words in insults',title_x=0.5,
                  template='plotly_dark')
fig.show()

# Apply LDA model for tweets¶

In [ ]:
from gensim.corpora import Dictionary

id2word = Dictionary(tweet_df_token)

texts =tweet_df_token

corpus = [id2word.doc2bow(text) for text in texts]

corpus[:1]

In [ ]:
from gensim.models import LdaModel
lda_model = LdaModel(corpus=corpus,
                    id2word=id2word,
                    num_topics=20, 
                    random_state=100,
                    update_every=1,
                    chunksize=100,
                    passes=10,
                    alpha='auto',
                    per_word_topics=True)

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

# Building a Random Forest Model

In [ ]:
Top_targets=most_Insulted.iloc[0:5,0]
Top_5=['the-media', 'democrats','hillary-clinton', 'trump-russia', 'joe-biden']

In [ ]:
df['Top_5'] = df.target.apply(lambda x : 1 if (x  in Top_5 ) else 0)
y=df['Top_5']
df

In [ ]:
#Tokenization using count vectorizer
count_vect = CountVectorizer(ngram_range=(1,1))
token = count_vect.fit_transform(tweet_df_lemma)
token

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(ngram_range=(1,1), min_df=0.05, max_df=0.95)
tfidf_vect

In [ ]:
token_tweet = tfidf_vect.fit_transform(tweet_df_lemma)

In [ ]:
df_rf= pd.DataFrame(token_tweet.toarray(), columns=tfidf_vect.get_feature_names())

In [ ]:
df_rf.shape

In [ ]:
from sklearn.model_selection import train_test_split

train_x, test_x,train_y, test_y = train_test_split(df_rf,y, test_size = 0.2, random_state = 5)
print(train_x.shape, test_x.shape)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {'n_estimators':[130,150,160,180,200],
              'max_features':[13,15,17,19]}

grid_rf = GridSearchCV(estimator= RandomForestClassifier(),
                      param_grid=param_grid,
                      cv = 10,
                      n_jobs=-1, verbose=True)

grid_rf.fit(train_x,train_y)

In [ ]:
grid_rf.best_score_

In [ ]:
grid_rf.best_params_

In [ ]:
grid_rf.best_estimator_

In [ ]:
#Fit the model
rf_model = grid_rf.best_estimator_
rf_model.fit(train_x, train_y)

In [ ]:
rf_train_predict = pd.DataFrame({'actual' : train_y,
                                 'predicted' : rf_model.predict(train_x)})
rf_train_predict.head()


In [ ]:
rf_test_predict = pd.DataFrame({'actual' : test_y,
                                 'predicted' : rf_model.predict(test_x)})
rf_test_predict.head()

# Model Evaluation

In [ ]:
print('Accuracy Score for train dataset : ' , metrics.accuracy_score(rf_train_predict.actual, rf_train_predict.predicted))
print('Accuracy Score for test dataset : ' , metrics.accuracy_score(rf_test_predict.actual, rf_test_predict.predicted))

In [ ]:
print('ROC-AUC Score for train dataset : ' , metrics.roc_auc_score(rf_train_predict.actual, rf_train_predict.predicted))
print('ROC-AUC Score for validation dataset : ' , metrics.roc_auc_score(rf_test_predict.actual, rf_test_predict.predicted))

In [ ]:
conn_cm_test = metrics.confusion_matrix(rf_test_predict.actual, rf_test_predict.predicted, [1,0])
sns.heatmap(conn_cm_test, fmt= '.2f', annot=True,  xticklabels=['Top 7', 'Top 7'], yticklabels=['Top 7', 'Top 7'])

In [ ]:
print(metrics.classification_report(rf_test_predict.actual, rf_test_predict.predicted))

In [ ]:
indices = np.argsort(rf_model.feature_importances_)[::-1]
feature_rank = pd.DataFrame(columns = ['rank', 'feature', 'importance'])
for f in range(train_x.shape[1]):
    feature_rank.loc[f] = [f+1,
                          train_x.columns[indices[f]],
                          rf_model.feature_importances_[indices[f]]]
feature_rank.round(3)

In [ ]:
feature_rank[:17]